In [62]:
import numpy 
import pandas as pd

import requests
from datetime import datetime, timezone
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
from base64 import b64encode

ROOT_URL = "https://192.168.1.24/piwebapi"
COMPUTER = "WIN-5MGCA5NMS7S"
DATABASE = "Database1"
USERNAME = f"{COMPUTER}\Administrator"
PASSWORD = "abcd1234.."

def basic_auth(username, password):
    token = b64encode(f"{username}:{password}".encode('utf-8')).decode("ascii")
    return f'Basic {token}'

BASIC_AUTH_TOKEN = basic_auth(USERNAME, PASSWORD)
POSTHEADERS = { 'X-Requested-With': 'True', 'Content-Type': 'application/json', 'Authorization': BASIC_AUTH_TOKEN }

# Test Server
response_server = requests.get(f"{ROOT_URL}/", auth=(USERNAME, PASSWORD), verify=False)
response_server.json()


/home/arkiven4/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.24'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'Links': {'Self': 'https://192.168.1.24/piwebapi/',
  'AssetServers': 'https://192.168.1.24/piwebapi/assetservers',
  'DataServers': 'https://192.168.1.24/piwebapi/dataservers',
  'Omf': 'https://192.168.1.24/piwebapi/omf',
  'System': 'https://192.168.1.24/piwebapi/system'}}

In [8]:
# Get Database WebID
elementPath = f"\\\\{COMPUTER}\\{DATABASE}"
rsp_element_raw = requests.get(f"{ROOT_URL}/assetdatabases?path={elementPath}", auth=(USERNAME, PASSWORD), verify=False)
DATABASE_WEBID = rsp_element_raw.json()['WebId']

/home/arkiven4/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.24'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [60]:
# Get DATASERVER WebID
elementPath = f"\\\\{COMPUTER}\\{DATABASE}"
rsp_element_raw = requests.get(f"{ROOT_URL}/dataservers", auth=(USERNAME, PASSWORD), verify=False)
DATASERVER_WEBID = rsp_element_raw.json()['Items'][0]['WebId']

/home/arkiven4/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.24'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [80]:
# Get By Path Element
elementPath = f"\\\\{COMPUTER}\\{DATABASE}\\Data_Site\\LGS"
rsp_element_raw = requests.get(f"{ROOT_URL}/elements?path={elementPath}", auth=(USERNAME, PASSWORD), verify=False)
rsp_element_json = rsp_element_raw.json()
ELEMENT_TARGET = rsp_element_json['WebId']

/home/arkiven4/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.24'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [82]:
# Create New Point #201 Created # 409 Duplicated
rsp_do_raw = requests.post(f"{ROOT_URL}/dataservers/{DATASERVER_WEBID}/points", headers=POSTHEADERS, data=json.dumps({
  "Name": "Coba1",
  "Descriptor": "Coba1",
  "PointClass": "classic",
  "PointType": "Float32",
  "EngineeringUnits": "",
  "Step": False,
  "Future": False,
  "DisplayDigits": -5
}), verify=False)

rsp_do_raw.headers['Location']
point_webid = rsp_do_raw.headers['Location'].split("/")[-1]

/home/arkiven4/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.24'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [201]>

In [76]:
# Create New Attribute
rsp_do_raw = requests.post(f"{ROOT_URL}/elements/{ELEMENT_TARGET}/attributes", headers=POSTHEADERS, data=json.dumps({
  "Name": "WaterEda2",
  "Description": "2008 Water Use",
  "Type": "Int32",
  "TypeQualifier": "",
  "DefaultUnitsName": "liter",
  "DisplayDigits": -5,
  "DataReferencePlugIn": "PI Point",
  "ConfigString": "Coba3",
  "IsConfigurationItem": False,
  "IsHidden": False,
  "IsManualDataEntry": False,
  "CategoryNames": [
    "Energy Savings Targets"
  ]
}), verify=False)

rsp_do_raw

/home/arkiven4/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.24'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [201]>

In [99]:
# Create New Attribute
rsp_do_raw = requests.post(f"{ROOT_URL}/streams/{point_webid}/recorded", headers=POSTHEADERS, data=json.dumps([
  {
    "Timestamp": "2024-05-01T02:15:00Z",
    "UnitsAbbreviation": "m",
    "Good": True,
    "Questionable": False,
    "Value": 3
  },
  {
    "Timestamp": "2024-05-01T01:45:00Z",
    "UnitsAbbreviation": "m",
    "Good": True,
    "Questionable": False,
    "Value": 2
  }
]), verify=False)

rsp_do_raw.text

/home/arkiven4/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.24'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


''

In [100]:
rsp_do_raw

<Response [202]>

In [91]:
WS = pd.read_excel('LGS#1_Overall 2017-2023 (hourly).xlsx', "LGS#1 2023")

In [92]:
WS.head()

,TimeStamp,Active Power (MW),Grid (Aux: 0 & FCE:1),Reactive Power (MVAr),Exc Voltage (Volt),Exc Current (A),V Phase 1 (Volt),V Phase 2 (Volt),V Phase 3 (Volt),Current Phase 1 (A),...,LGB Metal Temp 2,LGB Oil Temp,Gen air Cooler Air Otlet Temp,Gen air Cooler Water Inlet Temp,Gen air Cooler Water Otlet Temp,Gen Cooler Water Flow (LPM),UGB Cooling Water Flow (LPM),LGB Cooling Water Flow (LPM),Lower Canal Level (msAL),TailRace Level (msAL)
0,2023-01-01 00:00:01,37.61,0.0,19.07,123.82,702.02,10.77,10.73,10.84,0.0,...,53.0,70.0,55.14,61.01,59.31,3795.33,32.42,945.70,308.1,165.96
1,2023-01-01 01:00:01,37.51,0.0,19.04,123.86,704.99,10.77,10.73,10.83,0.0,...,53.0,70.0,55.26,61.63,60.00,3797.60,32.42,946.46,308.1,165.97
2,2023-01-01 02:00:01,37.34,0.0,19.00,123.34,702.22,10.76,10.72,10.83,0.0,...,53.0,70.0,55.39,62.00,60.00,3793.92,32.42,944.96,308.1,165.98
3,2023-01-01 03:00:02,37.17,0.0,18.97,124.49,703.04,10.76,10.72,10.82,0.0,...,53.0,70.0,55.51,62.00,60.00,3798.31,32.42,947.38,308.1,165.99
4,2023-01-01 04:00:02,37.01,0.0,18.93,121.03,702.86,10.75,10.72,10.82,0.0,...,53.0,70.0,55.64,62.00,60.00,3806.77,32.42,948.19,308.1,166.00
